In [1]:
%reload_ext watermark
%watermark -uniz --author "Prayson W. Daniel" -vm -p duckdb,pandas,polars,river,scikit-learn

Author: Prayson W. Daniel

Last updated: 2024-10-19T10:40:12.711192+02:00

Python implementation: CPython
Python version       : 3.9.6
IPython version      : 8.18.1

duckdb      : 1.1.2
pandas      : 2.2.3
polars      : 1.9.0
river       : 0.21.2
scikit-learn: 1.5.2

Compiler    : Clang 15.0.0 (clang-1500.3.9.4)
OS          : Darwin
Release     : 23.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit



### Show me the money ... data

In [2]:
%cd ..

/Users/pwd/Codes/hadithi/dev.io/movies


In [3]:
import duckdb
import polars as pl

In [4]:
MOVIE_QUERY: str = """
WITH movies AS (
     SELECT 
         movie_title AS title
         ,director_name AS director
         ,title_year AS year
         ,CONCAT(actor_1_name,'|',actor_2_name,'|', actor_3_name) AS actors
         ,GREATEST(actor_1_facebook_likes,actor_2_facebook_likes,actor_3_facebook_likes) AS max_fb_likes
         ,duration
         ,genres
         ,plot_keywords
         ,budget
         ,gross
         ,imdb_score AS imdb
         ,num_voted_users
         ,country
         ,content_rating
         ,ROW_NUMBER() OVER (PARTITION BY movie_title,director_name  ORDER BY title_year) AS row_num --used to remove duplicates

    FROM 'data/clean.csv'
    WHERE genres NOT IN ['Documentary', 'History', 'Biography', 'Sport']
        AND title_year > 1999
        AND language = 'English'
        AND LENGTH(director_name) <> 0
        AND LENGTH(content_rating) <> 0
        AND aspect_ratio <> 0
        AND budget <> 0
        AND gross <> 0
)
    SELECT 
      *
    FROM movies
    WHERE row_num = 1 --removes duplicates
        ORDER BY year DESC
"""

In [7]:
%%time

movies = duckdb.sql(MOVIE_QUERY).pl()
movies.sample(3)

CPU times: user 87.3 ms, sys: 27.8 ms, total: 115 ms
Wall time: 83.8 ms


title,director,year,actors,max_fb_likes,duration,genres,plot_keywords,budget,gross,imdb,num_voted_users,country,content_rating,row_num
str,str,i64,str,i64,i64,str,str,i64,i64,f64,i64,str,str,i64
"""Shooter ""","""Antoine Fuqua""",2007,"""Tate Donovan|Ned Beatty|Louis …",650,124,"""Action|Crime|Drama|Mystery|Thr…","""colonel|ethiopia|marksman|on t…",61000000,46975183,7.2,256213,"""USA""","""R""",1
"""Turbo ""","""David Soren""",2013,"""Ryan Reynolds|Snoop Dogg|Ben S…",16000,96,"""Adventure|Animation|Comedy|Fam…","""accident|freak accident|race|s…",135000000,83024900,6.5,62424,"""USA""","""PG""",1
"""Act of Valor ""","""Mike McCoy""",2012,"""Alex Veadov|Jason Cottle|Rorke…",93,110,"""Action|Adventure|Drama|Thrille…","""mission|navy seal|terrorist|to…",12000000,70011073,6.5,57996,"""USA""","""R""",1


In [8]:
%%time

# get all unique genres to explode them as columns
unique_genres = (
    movies.select(pl.col("genres").str.split("|").explode())
    .unique()
    .to_series()
    .to_list()
)


(
    movies.with_columns(
        [
            pl.col("genres")
            .str.contains(f"\\b{genre}\\b")  # TODO: Find a faster way
            .cast(pl.Int8)
            .alias(genre)
            for genre in unique_genres
        ]
    ).drop(pl.col("genres"))
).sample(3)

CPU times: user 11.2 ms, sys: 7.31 ms, total: 18.5 ms
Wall time: 6.16 ms


title,director,year,actors,max_fb_likes,duration,plot_keywords,budget,gross,imdb,num_voted_users,country,content_rating,row_num,Action,Animation,Thriller,Fantasy,Mystery,Biography,History,Sci-Fi,Western,Musical,Drama,Family,Music,Romance,Horror,War,Documentary,Adventure,Crime,Sport,Comedy
str,str,i64,str,i64,i64,str,i64,i64,f64,i64,str,str,i64,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8
"""The Nativity Story ""","""Catherine Hardwicke""",2006,"""Keisha Castle-Hughes|Hiam Abba…",446,101,"""angel|bethlehem|journey|nazare…",30000000,37617947,6.8,8143,"""USA""","""PG""",1,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0
"""Auto Focus ""","""Paul Schrader""",2002,"""Ed Begley Jr.|Michael McKean|K…",783,105,"""friendship|photography|radio|s…",7000000,2062066,6.6,11387,"""USA""","""R""",1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
"""The Day the Earth Stood Still ""","""Scott Derrickson""",2008,"""Keanu Reeves|Jon Hamm|Juan Rie…",18000,104,"""alien|earth|giant robot|milita…",80000000,79363785,5.5,139426,"""USA""","""PG-13""",1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
